In [1]:
import pandas as pd
from sklearn.model_selection import KFold, train_test_split
from sklearn.linear_model import LinearRegression, Ridge, ElasticNet, BayesianRidge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import VotingRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import RepeatedKFold, GridSearchCV
from sklearn.svm import SVR
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
plt.style.use('seaborn-whitegrid')
from tqdm import tqdm
import sys
from pvlib import solarposition
import sys
import os
sys.path.append(os.path.abspath('../common/'))
import common_functions_by_date as cfbd
import datetime
import predictions_algorithmes as pa

In [3]:
weather_path = 'Input/weather_train_set0.csv'
demand_path = 'Input/demand_train_set0.csv'
solar_path = 'Input/pv_train_set0.csv'
dp = cfbd.DataPreprocesser(weather_path, demand_path, solar_path)
dp.load_df()
dp.remove_nan()
dp.interpolate_df()
dp.get_zenith_angle()
dp.get_poa_and_ghi_irradiance()
dp.df.head()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,...,solar_location5,solar_location1,week,dow,date,hour,sp,zenith_angle,GHI,POA
datetime,,,,,,,,,,,,,,,,,,,,,
2017-11-03 00:00:00,2.19,0.0,0.0,7.05,7.46,13.20,9.640,6.680,13.090,8.560,...,0.0,0.0,44,4,2017-11-03,0,1.0,122.821452,0.0,0.0
2017-11-03 00:30:00,2.14,0.0,0.0,7.38,7.30,13.26,9.675,6.475,13.150,8.625,...,0.0,0.0,44,4,2017-11-03,0,2.0,115.780705,0.0,0.0
2017-11-03 01:00:00,2.01,0.0,0.0,7.70,7.14,13.32,9.710,6.270,13.210,8.690,...,0.0,0.0,44,4,2017-11-03,1,3.0,108.668070,0.0,0.0
2017-11-03 01:30:00,1.87,0.0,0.0,7.48,7.00,13.34,9.720,6.090,13.255,8.715,...,0.0,0.0,44,4,2017-11-03,1,4.0,101.505357,0.0,0.0
2017-11-03 02:00:00,1.86,0.0,0.0,7.20,6.86,13.36,9.730,5.910,13.300,8.740,...,0.0,0.0,44,4,2017-11-03,2,5.0,94.307845,0.0,0.0


In [3]:
dp.df.tail()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,...,solar_location5,solar_location1,week,dow,date,hour,sp,zenith_angle,GHI,POA
datetime,,,,,,,,,,,,,,,,,,,,,
2018-07-22 21:30:00,2.40,0.0,0.0,15.93,17.20,19.295,18.80,19.78,19.425,17.985,...,0.0,0.0,29,6,2018-07-22,21,44.0,160.540546,0.0,0.0
2018-07-22 22:00:00,2.15,0.0,0.0,15.49,17.01,19.140,18.43,19.35,19.270,17.790,...,0.0,0.0,29,6,2018-07-22,22,45.0,155.682778,0.0,0.0
2018-07-22 22:30:00,1.93,0.0,0.0,15.19,16.60,18.980,18.14,18.98,19.130,17.590,...,0.0,0.0,29,6,2018-07-22,22,46.0,149.826058,0.0,0.0
2018-07-22 23:00:00,1.82,0.0,0.0,15.06,16.19,18.820,17.85,18.61,18.990,17.390,...,0.0,0.0,29,6,2018-07-22,23,47.0,143.451067,0.0,0.0
2018-07-22 23:30:00,1.63,0.0,0.0,15.03,16.19,18.820,17.85,18.61,18.990,17.390,...,0.0,0.0,29,6,2018-07-22,23,48.0,136.790913,0.0,0.0


In [4]:
first_day_pred=datetime.datetime(2018,7,23).date()

In [5]:
mp = cfbd.MLPredictor(dp,first_day_pred)
mp.get_demand_previous_week()
mp.get_weather_prediction(weather_path)
mp.pred_demand_with_forecast_method_and_average_with_previous_weeks(dp.df, 'Output/rectify_forecast', first_day_pred, 'demand_MW', compute_forecast=True)
mp.predict_pv_power_smooth_and_square_irr()

week prediction with start day :  2018-07-23
  0%|          | 0/336 [00:00<?, ?it/s]2018-07-23 00:00:00
KNeighborsRegressor(n_neighbors=30)
100%|██████████| 336/336 [01:22<00:00,  4.08it/s]
week prediction with start day :  2018-07-16
  0%|          | 0/336 [00:00<?, ?it/s]2018-07-16 00:00:00
KNeighborsRegressor(n_neighbors=55)
100%|██████████| 336/336 [01:19<00:00,  4.23it/s]
week prediction with start day :  2018-07-09
  0%|          | 0/336 [00:00<?, ?it/s]2018-07-09 00:00:00
KNeighborsRegressor(n_neighbors=51)
100%|██████████| 336/336 [01:23<00:00,  4.02it/s]
week prediction with start day :  2018-07-02
  0%|          | 0/336 [00:00<?, ?it/s]2018-07-02 00:00:00
KNeighborsRegressor(n_neighbors=44)
100%|██████████| 336/336 [01:22<00:00,  4.09it/s]
week prediction with start day :  2018-06-25
  0%|          | 0/336 [00:00<?, ?it/s]2018-06-25 00:00:00
KNeighborsRegressor(n_neighbors=45)
100%|██████████| 336/336 [01:19<00:00,  4.24it/s]
smooth_phaze : -3
weather dephasage : 1
smooth_pha

In [5]:
mp = cfbd.MLPredictor(dp,first_day_pred)
mp.get_demand_previous_week()
mp.get_weather_prediction(weather_path)
mp.pred_demand_with_forecast_method_and_average_with_previous_weeks(dp.df, 'Output/rectify_forecast', first_day_pred, 'demand_MW', compute_forecast=False)
mp.predict_pv_power_smooth_and_square_irr()

smooth_phaze : -3
weather dephasage : 1
smooth_phaze : -3


In [6]:
bpd = cfbd.BatteryPowerDispatcher
B = bpd.get_all_dispatch_in_a_week(bpd,mp.predicted_df, first_day_pred)
B_final = bpd.format_dispatching_for_competition(B, mp.predicted_df.index)

In [7]:
B.sum()

20180723    0.000000e+00
20180724    1.554312e-15
20180725   -1.776357e-15
20180726   -2.664535e-15
20180727    2.220446e-16
20180728   -4.440892e-16
20180729    1.110223e-15
dtype: float64

In [8]:
B_final.to_csv('Output/BGBattery_last_model_set0.csv')

## Naive prediction

In [9]:
mp.get_demand_previous_week()
mp.get_solar_power_previous_week()

,week,dow,sp,hour,zenith_angle,GHI,POA,demand_MW,temp_location3,temp_location6,...,temp_location4,temp_location5,temp_location1,solar_location3,solar_location6,solar_location2,solar_location4,solar_location5,solar_location1,pv_power_mw
datetime,,,,,,,,,,,,,,,,,,,,,
2018-07-23 00:00:00,30.0,0.0,1.0,0.0,129.964392,0.0,0.0,1.73,15.670,18.500,...,17.860,18.740,16.740,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-23 00:30:00,30.0,0.0,2.0,0.0,123.037135,0.0,0.0,1.62,15.570,18.355,...,17.585,18.630,16.620,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-23 01:00:00,30.0,0.0,3.0,1.0,116.048656,0.0,0.0,1.56,15.470,18.210,...,17.310,18.520,16.500,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-23 01:30:00,30.0,0.0,4.0,1.0,109.024984,0.0,0.0,1.48,15.370,18.110,...,17.050,18.455,16.410,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-23 02:00:00,30.0,0.0,5.0,2.0,101.985047,0.0,0.0,1.48,15.270,18.010,...,16.790,18.390,16.320,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2018-07-29 21:30:00,30.0,6.0,44.0,21.0,161.775904,0.0,0.0,2.40,15.485,17.415,...,16.285,17.625,16.125,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-29 22:00:00,30.0,6.0,45.0,22.0,156.625250,0.0,0.0,2.15,15.310,17.390,...,16.030,17.580,16.040,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2018-07-29 22:30:00,30.0,6.0,46.0,22.0,150.539409,0.0,0.0,1.93,15.215,17.380,...,15.835,17.570,15.980,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [10]:
bpd = cfbd.BatteryPowerDispatcher
B = bpd.get_all_dispatch_in_a_week(bpd,mp.predicted_df, first_day_pred)
B_final = bpd.format_dispatching_for_competition(B, mp.predicted_df.index)

In [11]:
B_final.to_csv('Output/BGBattery_naive_set0.csv')

## Dispatch with real value

In [24]:
weather_path_task1 = '../task1/Input/weather_train_set1.csv'
demand_path_task1 = '../task1/Input/demand_train_set1.csv'
solar_path_task1 = '../task1/Input/pv_train_set1.csv'
dp_task1 = cfbd.DataPreprocesser(weather_path_task1, demand_path_task1, solar_path_task1)
dp_task1.load_df()
dp_task1.remove_nan()
dp_task1.interpolate_df()
# dp.get_zenith_angle()
dp_task1.get_poa_and_ghi_irradiance()
dp_task1.df.head()

,demand_MW,irradiance_Wm-2,pv_power_mw,panel_temp_C,temp_location3,temp_location6,temp_location2,temp_location4,temp_location5,temp_location1,...,solar_location4,solar_location5,solar_location1,week,dow,date,hour,sp,GHI,POA
datetime,,,,,,,,,,,,,,,,,,,,,
2017-11-03 00:00:00,2.19,0.0,0.0,7.05,7.46,13.20,9.640,6.680,13.090,8.560,...,0.0,0.0,0.0,44,4,2017-11-03,0,1.0,0.0,0.0
2017-11-03 00:30:00,2.14,0.0,0.0,7.38,7.30,13.26,9.675,6.475,13.150,8.625,...,0.0,0.0,0.0,44,4,2017-11-03,0,2.0,0.0,0.0
2017-11-03 01:00:00,2.01,0.0,0.0,7.70,7.14,13.32,9.710,6.270,13.210,8.690,...,0.0,0.0,0.0,44,4,2017-11-03,1,3.0,0.0,0.0
2017-11-03 01:30:00,1.87,0.0,0.0,7.48,7.00,13.34,9.720,6.090,13.255,8.715,...,0.0,0.0,0.0,44,4,2017-11-03,1,4.0,0.0,0.0
2017-11-03 02:00:00,1.86,0.0,0.0,7.20,6.86,13.36,9.730,5.910,13.300,8.740,...,0.0,0.0,0.0,44,4,2017-11-03,2,5.0,0.0,0.0


In [25]:
real_value_df = dp_task1.df[(dp_task1.df.index.date >= first_day_pred) & (dp_task1.df.index.date < first_day_pred + datetime.timedelta(days=7))]

In [26]:
bpd = cfbd.BatteryPowerDispatcher
B_real = bpd.get_all_dispatch_in_a_week(bpd,real_value_df, first_day_pred)
B_final_real = bpd.format_dispatching_for_competition(B_real, real_value_df.index)

In [27]:
B_real.sum()

20180723   -1.332268e-15
20180724   -1.110223e-15
20180725    2.220446e-16
20180726    4.218847e-15
20180727    0.000000e+00
20180728    6.661338e-16
20180729   -8.881784e-16
dtype: float64

In [28]:
B_final_real.to_csv('Output/BGBattery_real_set0.csv')